In [9]:
%reload_ext autoreload
%autoreload 2
import sys; sys.path.append('..')
from functools import partial
from src.data_utils import get_dataset_imagenet
from jupyter_preproc import smoothen, probs_to_softacc, nonmono, thresh, easy, normalize, get_entropy, accuracy
import numpy as np
import time
import os
from src.simple_utils import load_pickle

rng = np.random.default_rng(5)

In [17]:
G['all_acc']

array([0.01644242, 0.05137273, 0.07702424, 0.10272727, 0.13336364,
       0.15143939, 0.15297879, 0.16083333, 0.16600606, 0.17676667,
       0.18111212, 0.18632121, 0.18388485, 0.19758485, 0.21299091,
       0.21867576, 0.23102727, 0.22122121, 0.23330606, 0.29523939,
       0.30501818, 0.30855758, 0.3117303 , 0.31255152, 0.31384848,
       0.31229697, 0.31054848, 0.30488485, 0.30824848, 0.30711818,
       0.3270697 , 0.3305303 , 0.33325455, 0.33371515, 0.33529091,
       0.33691515, 0.33700606, 0.33601515, 0.33692424, 0.3348    ])

In [16]:
dname = 'imagenet_r'
G = load_pickle(f'../processed/imagenet/resnet50/resnet50_{dname}')
## get it into standard form (index ordering)
for k in ['probs', 'freqs', 'labels', 'labels_err', 'points', 'points_err']:
    G[k] = np.moveaxis(G[k], 0, -1)
    print(f'G[{k}] shape: {G[k].shape}')
    
outfname = f'../processed/imagenet/resnet50/resnet50_{dname}.npz'
print('saving to:', outfname)
start_time = time.time()
with open(outfname, 'wb') as f:
    np.savez(f, **G)
print('done: %f', (time.time() - start_time))

G[probs] shape: (30000, 200, 40)
G[freqs] shape: (30000, 200, 40)
G[labels] shape: (200, 40)
G[labels_err] shape: (200, 40)
G[points] shape: (30000, 40)
G[points_err] shape: (30000, 40)
saving to: ../processed/imagenet/resnet50/resnet50_imagenet_r.npz
done: %f 7.144935846328735


In [18]:
tname = 'imagenet'
dnames = ['imagenet_v2', 'imagenet_r', 'imagenet_a']
gnames = ['resnet50']

# smoothing parameters
num_gridpts = 50
smooth = True
sigma = 2.0

xkey = 'x'
Ykey = 'probs'

measures = {'ent': get_entropy}
funcs = [nonmono, thresh, easy]

for dname in dnames:
    dataset = get_dataset_imagenet(dname)
    ytest = dataset.targets
    measures['acc'] = partial(accuracy, ytest=ytest)
    
    for gname in gnames:
        print(f'{dname} {gname}')
        fname = f'../processed/{tname}/{gname}/{gname}_{dname}.npz'
        
        print('loading processed')
        G = np.load(fname)
        print('done\n')
        
        # G['xsoft'] = probs_to_softacc(G['probs'], ytest)
        
        print('smoothing')
        res = smoothen(G, xkey, Ykey)
        print('done\n')
        
        scores = dict()
        for meas_name, meas in measures.items():
            print(meas_name)
            m = meas(res[f'{Ykey}_smooth'])
            print(m.shape)
    
            for func in funcs:
                print(func.__name__)
                f_name = f'{meas_name}_{func.__name__}'
                s_vec = func(m)
                print(s_vec.shape)
                scores[f_name] = normalize(s_vec)
        
        os.makedirs(f'../scores/{tname}/{gname}/', exist_ok=True)
        outfname = f'../scores/{tname}/{gname}/{gname}_{dname}_scores.npz'
        
        print(f'saving to: {outfname}')                     
        with open(outfname, 'wb') as f:
            np.savez(f, **scores)
        print('done\n')

imagenet_v2 resnet50
loading processed
done

smoothing
done

ent
(10000, 50)
nonmono
(10000,)
thresh
(10000,)
easy
(10000,)
acc
(10000, 50)
nonmono
(10000,)
thresh
(10000,)
easy
(10000,)
saving to: ../scores/imagenet/resnet50/resnet50_imagenet_v2_scores.npz
done

imagenet_r resnet50
loading processed
done

smoothing
done

ent
(30000, 50)
nonmono
(30000,)
thresh
(30000,)
easy
(30000,)
acc
(30000, 50)
nonmono
(30000,)
thresh
(30000,)
easy
(30000,)
saving to: ../scores/imagenet/resnet50/resnet50_imagenet_r_scores.npz
done

imagenet_a resnet50
loading processed
done

smoothing
done

ent
(7500, 50)
nonmono
(7500,)
thresh
(7500,)
easy
(7500,)
acc
(7500, 50)
nonmono
(7500,)
thresh
(7500,)
easy
(7500,)
saving to: ../scores/imagenet/resnet50/resnet50_imagenet_a_scores.npz
done

